In [1]:
import sys
sys.path.insert(0, '../../modules')
get_ipython().magic(u'run ../visualizations.ipynb')
from hits import CDCHits, CTHHits, CyDetHits
from memory_profiler import memory_usage
from pprint import pprint
from collections import Counter, OrderedDict
%load_ext memory_profiler
from root_numpy import root2array
import scipy.sparse

Populating the interactive namespace from numpy and matplotlib


In [2]:
%run ../data_tools.ipynb

In [3]:
import hits
reload(hits)
from hits import CDCHits, CTHHits, CyDetHits

In [4]:
file_base = "/home/elg112/development/ICEDUST/track_finding_standalone/"\
            "track_finding_yandex/data/"
signal_files  = ["oa_xx_xxx_09100000-0000_opgjz6dop7ho_user-TrkTree_000_signal-label.root",
                 "oa_xx_xxx_09200000-0000_bxszqgjrri2n_user-TrkTree_000_signal-label.root",
                 "oa_xx_xxx_09400000-0000_7i52rtj3sz5v_user-TrkTree_000_signal-label.root",
                 "oa_xx_xxx_09600000-0000_xjij4zcypjul_user-TrkTree_000_signal-label.root",
                 "oa_xx_xxx_09800000-0000_b72ef5hzppor_user-TrkTree_000_signal-label.root",
                 "oa_xx_xxx_09900000-0000_orbwudhvihxn_user-TrkTree_000_signal-label.root"]

signal_files = ["oa_xx_xxx_09000000-0000_pdevedxn64ot_user-TrkTree_000_500signal-label.root",
                "oa_xx_xxx_09100000-0000_skzt2l7sdqvx_user-TrkTree_000_500signal-label.root",
                "oa_xx_xxx_09200000-0000_x67rltrbhc3n_user-TrkTree_000_500signal-label.root",
                "oa_xx_xxx_09300000-0000_seae7w5z6fvt_user-TrkTree_000_500signal-label.root",
                "oa_xx_xxx_09400000-0000_o774i4trtmim_user-TrkTree_000_500signal-label.root",
                "oa_xx_xxx_09500000-0000_ufkycd74hb4i_user-TrkTree_000_500signal-label.root",
                "oa_xx_xxx_09600000-0000_arxgz3idch3j_user-TrkTree_000_500signal-label.root",
                "oa_xx_xxx_09700000-0000_5xz67bjkzokx_user-TrkTree_000_500signal-label.root",
                "oa_xx_xxx_09800000-0000_jrdukg5g2ivh_user-TrkTree_000_500signal-label.root",
                "oa_xx_xxx_09900000-0000_kgij2afd5rtp_user-TrkTree_000_500signal-label.root"]

signal_files = ["oa_xx_xxx_09000000-0000_qs4c2zivfkyl_user-TrkTree_000_700signal-label.root",
                "oa_xx_xxx_09100000-0000_k6i42km533k7_user-TrkTree_000_700signal-label.root",
                "oa_xx_xxx_09200000-0000_uuu3dtsod77t_user-TrkTree_000_700signal-label.root",
                "oa_xx_xxx_09300000-0000_2s4hlrntpcyh_user-TrkTree_000_700signal-label.root",
                "oa_xx_xxx_09400000-0000_vrw7y5jhefms_user-TrkTree_000_700signal-label.root",
                "oa_xx_xxx_09500000-0000_2xxlflqodbqq_user-TrkTree_000_700signal-label.root",
                "oa_xx_xxx_09600000-0000_ak6psvrdnn4v_user-TrkTree_000_700signal-label.root",
                "oa_xx_xxx_09700000-0000_bipfrfikeaqy_user-TrkTree_000_700signal-label.root",
                "oa_xx_xxx_09800000-0000_5w5tj34s5elw_user-TrkTree_000_700signal-label.root",
                "oa_xx_xxx_09900000-0000_diro4pagpbtt_user-TrkTree_000_700signal-label.root"]


back_files = ["batch_2_TrkTree_2500.root",
              "batch_3_TrkTree_2536.root",
              "batch_4_TrkTree_5250.root"]

signal_files = [ file_base + file_name for file_name in signal_files]
back_files   = [ file_base + file_name for file_name in back_files]

In [5]:
these_branches = {}
these_branches["CDC"] = ["MCPos.fP.fX", "MCPos.fP.fY", "MCPos.fP.fZ"]
these_branches["CTH"] = ["MCPos.fP.fX", "MCPos.fP.fY", "MCPos.fP.fZ"]

In [6]:
test = data_import_sample(signal_files[1], back_files[1], 
                      these_cuts=["Trig","Track","700"],
                      branches=these_branches)
test.cth.set_trigger_time()


Using cuts
{ 'CDC': '1 == 1 && 1 == 1 && CDCHit.fDetectedTime < 1620 && CDCHit.fDetectedTime > 700',
  'CTH': '1 == 1 && 1 == 1 && CTHHit.fMCPos.fE < 1620 && CTHHit.fMCPos.fE > 700'}
Getting branches
{'CDC': ['MCPos.fP.fX', 'MCPos.fP.fY', 'MCPos.fP.fZ'],
 'CTH': ['MCPos.fP.fX', 'MCPos.fP.fY', 'MCPos.fP.fZ']}
Using cuts
{ 'CDC': 'CDCHit.fGoodTrig == 1 && CDCHit.fGoodTrack == 1 && CDCHit.fDetectedTime < 1620 && CDCHit.fDetectedTime > 700',
  'CTH': 'CTHHit.fGoodTrig == 1 && CTHHit.fGoodTrack == 1 && CTHHit.fMCPos.fE < 1620 && CTHHit.fMCPos.fE > 700'}
Getting branches
{'CDC': ['MCPos.fP.fX', 'MCPos.fP.fY', 'MCPos.fP.fZ'],
 'CTH': ['MCPos.fP.fX', 'MCPos.fP.fY', 'MCPos.fP.fZ']}
Trimming Signal Events
CTH Sig Events 2525 
CTH Back Events 2525 
CDC Sig Events 2525 
CDC Back Events 2525 


In [7]:
data_remove_coincidence(test)

In [8]:
def test_get_measurement(samp, events, verbose=False):
    return_value = True
    for evt in events:
        all_types = samp.cdc.get_events(events=evt)[samp.cdc.hit_type_name]
        all_types_2 = get_measurement(samp.cdc, samp.cdc.hit_type_name, events=evt)
        good = np.array_equal(all_types, all_types_2)
        if not good:
            print "Event: {}".format(evt)
            if verbose:
                print all_types.shape
                print all_types_2.shape
                print sum(samp.cdc.event_to_n_hits[evt])
                print samp.cdc.event_to_hits[evt].shape
            return_value = False
    return return_value

In [9]:
def get_measurement(self, name, events=None, shift=None, default=0,
                    only_hits=True, flatten=False, use_sparse=False):
    """
    Returns requested measurement in volumes, returning the default value if
    the hit does not register this measurement

    NOTE: IF COINCIDENCE HASN'T BEEN DEALT WITH THE FIRST HIT ON THE CHANNEL
    WILL BE TAKEN.  The order is determined by the hits_index

    :return: numpy.array of shape [len(events), CDC.n_points]
    """
    # Check if events is empty
    if events is None:
        events = np.arange(self.n_events)
    # Get the events as an array
    my_events = np.array(events)
    # Select the relevant event from data
    meas = self.get_events(my_events)[name]
    # Get the wire_ids and event_ids of the hit data
    wire_ids = self.get_hit_vols(my_events, unique=False)
    print len(wire_ids)
    # Map the evnt_ids to the minimal continous set
    evnt_ids = np.repeat(np.arange(my_events.size),
                         self.event_to_n_hits[my_events])
    print evnt_ids.shape
    # Get the default values
    result = default*np.ones((my_events.size, self.geom.n_points),
                              dtype=float)
    print result.shape
    # Figure out the first place each wire id is mentioned in each event
    two_d_ids = np.vstack([evnt_ids, wire_ids]).T
    print two_d_ids.shape
    # Get the unique evt_id/wire_id values
    one_d_view = np.ascontiguousarray(two_d_ids).view(
        np.dtype((np.void, two_d_ids.dtype.itemsize * two_d_ids.shape[1])))
    print one_d_view.shape
    return one_d_view, two_d_ids
    # Figure out where they appear
    _, first_hits = np.unique(one_d_view, return_index=True)
    print first_hits.shape
    # Map their appearances back in the correct order
    evnt_ids, wire_ids = two_d_ids[np.sort(first_hits)].T
    meas = meas[np.sort(first_hits)]
    print meas.shape
    # Put the result on the correct wires
    result[evnt_ids, wire_ids] = meas
    if shift is not None:
        result = result[:, self.geom.shift_wires(shift)]
    if only_hits:
        result = result[evnt_ids, wire_ids]
    if flatten:
        result = result.flatten()
    if use_sparse:
        return result.tocsr()
    return result


In [48]:
all_events = np.sort(np.arange(test.n_events)[:9*test.n_events/10])
np.random.shuffle(all_events)
all_events = np.sort(all_events)


test_evts = all_events
test_get_measurement(test, test_evts)

670
(670,)
(1, 4482)
(670, 2)
(670, 1)
Event: 0
466
(466,)
(1, 4482)
(466, 2)
(466, 1)
Event: 1
399
(399,)
(1, 4482)
(399, 2)
(399, 1)
Event: 2
556
(556,)
(1, 4482)
(556, 2)
(556, 1)
Event: 3
553
(553,)
(1, 4482)
(553, 2)
(553, 1)
Event: 4
604
(604,)
(1, 4482)
(604, 2)
(604, 1)
Event: 5
589
(589,)
(1, 4482)
(589, 2)
(589, 1)
Event: 6
545
(545,)
(1, 4482)
(545, 2)
(545, 1)
Event: 7
624
(624,)
(1, 4482)
(624, 2)
(624, 1)
Event: 8
683
(683,)
(1, 4482)
(683, 2)
(683, 1)
Event: 9
623
(623,)
(1, 4482)
(623, 2)
(623, 1)
Event: 10
621
(621,)
(1, 4482)
(621, 2)
(621, 1)
Event: 11
611
(611,)
(1, 4482)
(611, 2)
(611, 1)
Event: 12
722
(722,)
(1, 4482)
(722, 2)
(722, 1)
Event: 13
583
(583,)
(1, 4482)
(583, 2)
(583, 1)
Event: 14
492
(492,)
(1, 4482)
(492, 2)
(492, 1)
Event: 15
607
(607,)
(1, 4482)
(607, 2)
(607, 1)
Event: 16
793
(793,)
(1, 4482)
(793, 2)
(793, 1)
Event: 17
512
(512,)
(1, 4482)
(512, 2)
(512, 1)
Event: 18
595
(595,)
(1, 4482)
(595, 2)
(595, 1)
Event: 19
390
(390,)
(1, 4482)
(390, 2)


False

In [62]:
values = test.cdc.event_index_name
events = all_events
all_types = test.cdc.get_measurement(values, events=events)
all_types_2 = np.concatenate([test.cdc.get_measurement(values, 
                                                      events=evt) for evt in events])

In [63]:
print all_types.shape, len(np.unique(all_types))
print all_types_2.shape, len(np.unique(all_types_2))

(1296260,) 2272
(1296260,) 2272


In [61]:
print Counter(all_types)
print Counter(all_types_2)

Counter({26.0: 811, 83.0: 804, 17.0: 793, 29.0: 787, 80.0: 761, 13.0: 722, 64.0: 700, 35.0: 693, 73.0: 690, 9.0: 683, 62.0: 680, 71.0: 680, 0.0: 670, 86.0: 660, 70.0: 656, 55.0: 652, 88.0: 647, 63.0: 646, 41.0: 629, 43.0: 629, 8.0: 624, 10.0: 623, 22.0: 622, 53.0: 622, 11.0: 621, 12.0: 611, 72.0: 608, 16.0: 607, 31.0: 606, 85.0: 605, 5.0: 604, 59.0: 603, 68.0: 598, 19.0: 595, 6.0: 589, 14.0: 583, 49.0: 582, 77.0: 581, 66.0: 579, 47.0: 576, 78.0: 574, 67.0: 573, 97.0: 573, 61.0: 572, 93.0: 571, 32.0: 568, 23.0: 567, 37.0: 563, 57.0: 562, 75.0: 562, 89.0: 560, 25.0: 557, 30.0: 557, 3.0: 556, 4.0: 553, 74.0: 552, 94.0: 552, 79.0: 551, 65.0: 546, 7.0: 545, 99.0: 545, 46.0: 543, 52.0: 543, 28.0: 538, 84.0: 538, 56.0: 535, 27.0: 525, 69.0: 525, 91.0: 523, 96.0: 523, 21.0: 515, 98.0: 515, 18.0: 512, 82.0: 510, 36.0: 506, 33.0: 505, 76.0: 503, 81.0: 503, 60.0: 500, 24.0: 499, 90.0: 493, 15.0: 492, 87.0: 492, 39.0: 488, 48.0: 487, 58.0: 486, 34.0: 485, 95.0: 476, 51.0: 467, 1.0: 466, 44.0: 465,

In [58]:
values = test.cdc.event_index_name
events = all_events[1:3]
all_types_one_d, all_types_two_d = get_measurement(test.cdc, values, events=events)
#all_types_one_d_2 = np.vstack([get_measurement(test.cdc,
#                                                    values, 
#                                                    events=evt) for evt in events])

865
(865,)
(2, 4482)
(865, 2)
(865, 1)


In [53]:
print all_events[events], test.cdc.event_to_n_hits[events]

[1 2] [466 399]


In [54]:
from pandas import DataFrame

In [32]:
hit_vols = test.cdc.get_hit_vols(events, unique=False)
print hit_vols.shape
these_hits = hit_vols[:test.cdc.event_to_n_hits[events[0]]]
bad_ele = np.where(np.unique(these_hits, return_counts=True)[1] == 2)
bad_idx = np.unique(these_hits, return_index=True)[1][bad_ele]
rep_value = these_hits[bad_idx][0]
bad_elems = np.where(these_hits == rep_value)
print bad_elems, rep_value
print np.where(test.cdc.get_events(events[0])[test.cdc.flat_name] == 1195)
DataFrame(test.cdc.get_events(events)[540:600])

(1044,)


IndexError: index 0 is out of bounds for axis 0 with size 0

In [208]:
print sum(np.unique(test.cdc.get_hit_vols(3, unique=False), return_counts=True)[1] == 2).shape

()


In [143]:
bad_val = 2
bad_ele = numpy.where(numpy.unique(all_types_one_d, return_counts=True)[1] == bad_val)
bad_idx = numpy.unique(all_types_one_d, return_index=True)[1][bad_ele]
rep_val = all_types_one_d[bad_idx][0]
bad_elems = np.where(all_types_one_d[:,0] == rep_val)
print bad_val, bad_ele, bad_idx, rep_val, bad_elems
print all_types_two_d[bad_elems]

2 (array([377]),) [408] [        �      ] (array([408, 559]),)
[[   0 1195]
 [   0 1195]]


In [114]:
print np.unique(all_types_two_d[:,0])

[0 1]


In [69]:
print all_types_two_d.dtype.itemsize

8


In [74]:
print all_types_two_d.shape
print np.array_equal(all_types_two_d, np.ascontiguousarray(all_types_two_d))

(1109, 2)
True


In [77]:
print all_types_two_d[0:100].view(dtype=np.void)

[[         /      ]
 [               ]
 [         ,       ]
 [         +       ]
 [         �       ]
 [         �       ]
 [         �      ]
 [         �      ]
 [         �      ]
 [         �      ]
 [         f      ]
 [         e      ]
 [         d      ]
 [         c      ]
 [         b      ]
 [         ?      ]
 [         >      ]
 [         =      ]
 [         <      ]
 [         ;      ]
 [         :      ]
 [         9      ]
 [         Z      ]
 [         Y      ]
 [         X      ]
 [         W      ]
 [         V      ]
 [         |      ]
 [         {      ]
 [         �      ]
 [         �      ]
 [         �      ]
 [         �       ]
 [         �       ]
 [                ]
 [                ]
 [         �       ]
 [         �       ]
 [         �      ]
 [         �      ]
 [         _      ]
 [         `      ]
 [         :      ]
 [         ;      ]
 [               ]
 [               ]
 [         <      ]
 [         